In [12]:

import pandas as pd
import plotly.express as px
import ipywidgets as w
from IPython.display import display
import warnings

warnings.filterwarnings("ignore")

# ── Load data ───────────────────────────────────────────────────
only_sold = pd.read_csv("only_sold.csv")
sire_data = pd.read_csv("sire_data.csv")

# ── Widgets Shared by Plots ─────────────────────────────────────
all_sires = sorted(only_sold["Sire"].unique())
sire_search = w.Text(placeholder="Search sire…", layout=w.Layout(width="50%"))
sire_multiselect = w.SelectMultiple(options=all_sires, rows=8, layout=w.Layout(width="50%"))
data_toggle = w.ToggleButtons(options=["Excluding >95th Percentile", "Full"], description="Data:")

foal_min = w.IntText(value=10, description="Foals per year ≥", step=1)
yr_min, yr_max = int(sire_data.years_active.min()), int(sire_data.years_active.max())
year_range = w.IntRangeSlider(
    value=[yr_min, yr_max], min=yr_min, max=yr_max, step=1,
    description="Years active", layout=w.Layout(width="75%")
)

# ── Plot Functions ───────────────────────────────────────────────
def show_box_plot():
    subset_df = only_sold[only_sold.Price <= only_sold.Price.quantile(.95)]
    df = subset_df if data_toggle.value.startswith("Excluding") else only_sold
    if sire_multiselect.value:
        df = df[df["Sire"].isin(sire_multiselect.value)]

    fig = px.box(
        df, x="sale_year", y="Price",
        hover_data=["Sire", "Description", "Purchaser"],
        title="Keeneland Sept Yearling Sales by Sire"
    )
    fig.show()

def show_scatter_plot():
    lo, hi = year_range.value
    df = sire_data[(sire_data.foals_per_year >= foal_min.value) &
                   (sire_data.years_active.between(lo, hi))]

    fig = px.scatter(
        df.reset_index(), x="gini_coef", y="median_price",
        size="foals_per_year", color="foals_per_year",
        hover_name="Sire", color_continuous_scale="plasma",
        title="Sire scatter (interactive thresholds)"
    )
    fig.show()

def show_correlation_plot():
    lo, hi = year_range.value
    df = sire_data[(sire_data.foals_per_year >= foal_min.value) &
                   (sire_data.years_active.between(lo, hi))]

    corr_by_year = (
        df.groupby("years_active")
          .apply(lambda g: g["gini_coef"].corr(g["median_price"]))
          .dropna()
          .reset_index(name="corr")
          .sort_values("years_active")
    )

    fig = px.line(
        corr_by_year, x="years_active", y="corr",
        markers=True,
        title="Correlation (gini coef ↔ median price) by years active"
    )
    fig.update_layout(yaxis_title="Correlation [-1,1]", xaxis_title="Years active")
    fig.show()

# ── Reactive Filtering ──────────────────────────────────────────
def _filter_sires(_):
    pat = sire_search.value.strip().lower()
    keep = sire_multiselect.value
    opts = [o for o in all_sires if pat in o.lower()]
    sire_multiselect.options = opts
    sire_multiselect.value = tuple(o for o in keep if o in opts)

sire_search.observe(_filter_sires, names="value")

# ── Toggle Buttons to Switch Views ──────────────────────────────
plot_picker = w.ToggleButtons(options=["Yearly Sales Data", "Sire Performance", "Correlation"], description="View:")
plot_out = w.Output()

def render_plot(change):
    with plot_out:
        plot_out.clear_output(wait=True)
        if change["new"] == "Yearly Sales Data":
            display(w.VBox([data_toggle, w.HTML("<b>Sire filter</b>"), sire_search, sire_multiselect]))
            show_box_plot()
        elif change["new"] == "Sire Performance":
            display(w.HBox([foal_min, year_range]))
            show_scatter_plot()
        elif change["new"] == "Correlation":
            display(w.HBox([foal_min, year_range]))
            show_correlation_plot()

plot_picker.observe(render_plot, names="value")

# ── Display Full App ────────────────────────────────────────────
display(w.VBox([plot_picker, plot_out]))
plot_picker.value = "Yearly Sales Data"
